In [5]:
%load_ext autoreload
%autoreload 2


%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
%cd C:\MAD4AG

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

C:\MAD4AG


In [7]:
combined_df= pd.read_pickle(r'.\dbs\twins\multiple_days.pkl')


df_clusters=pd.read_parquet(f'./dbs/intermediate/df_selected_clusters.parquet')

df_clusters = df_clusters[df_clusters.uid.isin(combined_df.uid)]


In [8]:
# calculate mean and median of the number_of_other_cluster in MAD data


df_clusters_statistic= df_clusters[df_clusters.act_type=='other'].groupby(['uid']).size().reset_index(name='number_of_other_cluster')

df_clusters_statistic = pd.merge(combined_df[['uid']], df_clusters_statistic, on='uid', how='left')
df_clusters_statistic.fillna(0, inplace=True)

print('mean: ', df_clusters_statistic.number_of_other_cluster.mean())
print('median: ', df_clusters_statistic.number_of_other_cluster.median())
print('the share of people more than 10 other clusters: ',len(df_clusters_statistic[df_clusters_statistic.number_of_other_cluster>10])/len(df_clusters_statistic))
df_clusters_statistic.number_of_other_cluster.value_counts()

mean:  6.574893589054653
median:  3.0
the share of people more than 10 other clusters:  0.18115956036139128


1.0      57089
2.0      43543
3.0      28524
4.0      20042
5.0      15291
         ...  
142.0        1
331.0        1
137.0        1
139.0        1
146.0        1
Name: number_of_other_cluster, Length: 161, dtype: int64

In [9]:
file_name = f'./dbs/intermediate/stops_1_new.parquet'

#Read each batchs file
df = pd.read_parquet(file_name)
df = df[df.holiday_s != 1]
df = df[df.weekday_s == 1]
df = df.drop(['holiday_s', 'weekday_s'], axis=1)



In [10]:
df_clusters_freq = df.groupby(['uid','cluster']).size().reset_index(name='clusters_freq')

In [11]:
df_clusters = pd.merge(df_clusters, df_clusters_freq, on=['uid','cluster'], how='left')

In [12]:
df_clusters_other = df_clusters[df_clusters.act_type=='other']

In [13]:
df_clusters_other['freq_sum'] = df_clusters_other.groupby(['uid'])['clusters_freq'].transform("sum")

C:\Users\aglar\AppData\Local\anaconda3\envs\pycharm\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
df_clusters_other['other_prob'] = df_clusters_other['clusters_freq']/df_clusters_other['freq_sum']

C:\Users\aglar\AppData\Local\anaconda3\envs\pycharm\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
df_clusters_list= df_clusters_other.groupby(['uid'])['cluster'].apply(list).reset_index(name='clusters')

df_other_prob_list = df_clusters_other.groupby(['uid'])['other_prob'].apply(list).reset_index(name='other_prob')

In [16]:
df_other = pd.merge(combined_df[['uid', 'other_count_0', 'other_count_1', 'other_count_2', 'other_count_3', 'other_count_4']],  df_clusters_list, on='uid', how='left')
df_other = pd.merge(df_other, df_other_prob_list, on='uid', how='left')

In [17]:
df_other = df_other.dropna(subset=['clusters'])

In [18]:
def other_finder(data , i):
    other_clusters_place = np.argmax(np.random.multinomial(1, data['other_prob'], data['other_count_'+ str(i)]), axis=1).tolist()
    clusters_list = data['clusters']
    other_clusters = [clusters_list[place] for place in other_clusters_place]
    return other_clusters

In [19]:
for i in range(5):
    column_name= 'other_clusters_'+str(i)
    df_other[column_name] = df_other.apply(lambda row: other_finder(row, i), axis=1)

In [96]:
df_clusters_duration = df.groupby(['uid','cluster'])['dur'].sum().reset_index(name='total_duration')

In [97]:
df_clusters = pd.merge(df_clusters, df_clusters_duration, on=['uid','cluster'], how='left')


In [98]:
df_ppl_duration = df_clusters[df_clusters.act_type== 'other'].groupby(['uid'])['total_duration'].sum().reset_index(name='other_sum')

In [99]:
df_clusters = pd.merge(df_clusters, df_ppl_duration, on=['uid'], how='left')

In [100]:
df_clusters['other_prob'] = df_clusters['total_duration']/df_clusters['other_sum']

In [101]:
df_clusters['other_prob'][df_clusters.act_type.isin(['home','work','school'])]=0

C:\Users\aglar\AppData\Local\anaconda3\envs\pycharm\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [102]:
df_clusters_list= df_clusters.groupby(['uid'])['cluster'].apply(list).reset_index(name='clusters')

df_other_prob_list = df_clusters.groupby(['uid'])['other_prob'].apply(list).reset_index(name='other_prob')

here

In [103]:
#df_other = pd.merge(combined_df[['uid', 'act_seq_simp_0','other_count_0']][combined_df.other_count_0>0], df_clusters_list, on='uid', how='left')

In [118]:
df_other = pd.merge(combined_df[['uid','other_count_0','other_count_1','other_count_2','other_count_3','other_count_4']], df_clusters_list, on='uid', how='left')

In [119]:
df_other = pd.merge(df_other, df_other_prob_list, on='uid', how='left')

In [115]:
# df_other['other_clusters_place'] = df_other.apply( lambda row: np.argmax(np.random.multinomial(1, row['other_prob'], row['other_count_0']), axis=1).tolist()  , axis=1)

In [116]:
# def other_finder(data):
#     other_clusters_place = np.argmax(np.random.multinomial(1, data['other_prob'], data['other_count_0']), axis=1).tolist()
#     clusters_list = data['clusters']
#     data['other_clusters'] = [clusters_list[place] for place in other_clusters_place]
#     return data
#

In [112]:
# df_other = df_other.apply( lambda row: other_finder(row)  , axis=1)

In [120]:
def other_finder(data , i):
    other_clusters_place = np.argmax(np.random.multinomial(1, data['other_prob'], data['other_count_'+ str(i)]), axis=1).tolist()
    clusters_list = data['clusters']
    other_clusters = [clusters_list[place] for place in other_clusters_place]
    return other_clusters


In [121]:
for i in range(5):
    column_name= 'other_clusters_'+str(i)
    df_other[column_name] = df_other.apply(lambda row: other_finder(row, i), axis=1)